## Responses API란 무엇인가요?

Responses API는 OpenAI 모델과 상호작용하는 새로운 방식으로, 이전 API보다 더 간단하고 유연하게 설계되었습니다. 이 API는 여러 도구를 사용하고, 다중 턴 대화를 처리하며, 텍스트뿐만 아니라 다양한 유형의 데이터를 다룰 수 있는 고급 AI 애플리케이션을 쉽게 구축할 수 있도록 합니다.

Chat Completions와 같이 주로 텍스트를 위해 설계된 이전 API나 설정이 많이 필요한 Assistants API와 달리, Responses API는 다음을 위해 처음부터 설계되었습니다:

- 원활한 다중 턴 상호작용 (단일 API 호출에서 여러 단계의 대화를 이어갈 수 있음)
- 강력한 호스팅 도구에 대한 쉬운 접근 (파일 검색, 웹 검색, 코드 해석기 등)
- 모델에 보낼 컨텍스트에 대한 세밀한 제어

AI 모델이 점점 더 복잡하고 장기적인 추론을 수행할 수 있게 되면서, 비동기적이고 상태를 유지할 수 있는 API가 필요합니다. Responses API는 이러한 요구를 충족하도록 설계되었습니다.

이 가이드에서는 Responses API가 제공하는 새로운 기능과 함께 시작하는 데 도움이 되는 실용적인 예제를 살펴봅니다.

## 기본 사항

Responses API는 표면적으로 Completions API와 매우 유사하게 설계되었습니다.

In [3]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2025-04-14"
)

In [4]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="농담 하나 해줘",
)

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [ ]:
print(response.output[0].content[0].text)

Responses API의 주요 기능 중 하나는 상태를 유지한다는 점입니다. 즉, 대화의 상태를 직접 관리할 필요가 없으며, API가 이를 대신 처리합니다. 예를 들어, 언제든지 응답을 가져오면 전체 대화 기록이 포함됩니다.

In [ ]:
fetched_response = client.responses.retrieve(
    response_id=response.id
)

print(fetched_response.output[0].content[0].text)

이전 응답을 참조하여 대화를 계속할 수 있습니다.

In [ ]:
response_two = client.responses.create(
    model="gpt-4o-mini",
    input="다른 농담 하나 더 해줘",
    previous_response_id=response.id
)

In [ ]:
print(response_two.output[0].content[0].text)

물론 컨텍스트를 직접 관리할 수도 있습니다. 하지만 OpenAI가 컨텍스트를 유지해 주는 이점 중 하나는 응답을 원하는 시점에서 분기(fork)하여 해당 지점에서 대화를 계속할 수 있다는 점입니다.

In [ ]:
response_two_forked = client.responses.create(
    model="gpt-4o-mini",
    input="그 농담은 별로였어. 다른 농담 하나 더 해주고 두 농담의 차이점도 알려줘",
    previous_response_id=response.id  # 첫 번째 응답에서 분기하여 계속 진행
)

output_text = response_two_forked.output[0].content[0].text
print(output_text)

## 호스팅 도구

Responses API의 또 다른 이점은 `file_search` 및 `web_search`와 같은 호스팅 도구를 지원한다는 점입니다. 도구를 수동으로 호출하는 대신, 도구를 전달하면 API가 어떤 도구를 사용할지 결정하고 이를 사용합니다.

다음은 `web_search` 도구를 사용하여 웹 검색 결과를 응답에 통합하는 예제입니다.

In [ ]:
# web_search 도구는 특정 모델에서만 지원됩니다

client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2025-04-14"  # 최신 API 버전
)

# 💡 web_search를 사용하려면:
# 1. Azure Portal에서 gpt-4o (최신) 모델 배포
# 2. 또는 web_search를 지원하는 다른 모델 사용
# 3. 지원되는 리전(예: East US, West US)에서 배포 필요

In [ ]:
import json
print(json.dumps(response.output, default=lambda o: o.__dict__, indent=2))

In [ ]:
# web_search는 현재 Azure OpenAI의 특정 모델에서만 지원됩니다
# 대안: web_search 없이 일반 응답 생성 테스트

try:
    response = client.responses.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        input="AI에 대한 최신 뉴스는 무엇인가요?",
        # tools=[{"type": "web_search"}]  # 현재 지원되지 않음
    )
    print("✅ 응답 생성 성공 (web_search 없이)")
    print(f"\n응답: {response.output[0].content[0].text[:200]}...")
except Exception as e:
    print(f"❌ 에러: {e}")
    print("\n💡 참고: web_search 도구는 특정 Azure OpenAI 모델과 지역에서만 사용 가능합니다.")

## 멀티모달, 도구 확장 대화

Responses API는 텍스트, 이미지, 오디오 모달리티를 기본적으로 지원합니다.  
모든 것을 결합하여 Responses API를 통해 단일 API 호출로 완전한 멀티모달, 도구 확장 상호작용을 구축할 수 있습니다.

In [ ]:
import base64

from IPython.display import Image, display

# 제공된 URL에서 이미지를 표시
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"
display(Image(url=url, width=400))

response_multimodal = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": 
                 "이미지와 관련된 키워드를 생성하고, 검색 도구를 사용하여 키워드와 관련된 뉴스를 웹에서 검색하세요. "
                 "결과를 요약하고 출처를 인용하세요."},
                {"type": "input_image", "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"}
            ]
        }
    ],
    tools=[
        {"type": "web_search"}
    ]
)

In [ ]:
import json
print(json.dumps(response_multimodal.__dict__, default=lambda o: o.__dict__, indent=4))

위 예제에서는 `web_search` 도구를 사용하여 단일 API 호출로 이미지와 관련된 뉴스를 검색할 수 있었습니다. 이는 Chat Completions API를 사용할 경우 여러 번의 왕복 호출이 필요했던 작업을 단순화합니다.

Responses API를 사용하면  
🔥 단일 API 호출로 다음 작업을 처리할 수 있습니다:

✅ 멀티모달 입력을 사용하여 주어진 이미지를 분석합니다.

✅ `web_search` 호스팅 도구를 통해 웹 검색을 수행합니다.

✅ 결과를 요약합니다.

반면, Chat Completions API를 사용하면 여러 단계가 필요하며, 각 단계는 API로의 왕복 호출을 요구합니다:

1️⃣ 이미지를 업로드하고 분석 결과를 얻음 → 1회 요청

2️⃣ 정보를 추출하고 외부 웹 검색 호출 → 수동 단계 + 도구 실행

3️⃣ 도구 결과를 요약을 위해 다시 제출 → 추가 요청

다음 다이어그램에서 시각적으로 비교한 내용을 확인하세요!

![Responses vs Completions](../../images/comparisons.png)

Responses API를 사용해 보고 코드 단순화와 복잡한 멀티모달, 도구 확장 상호작용을 구축하는 데 얼마나 유용한지 확인해 보세요!

## Responses API란 무엇인가요?

Responses API는 OpenAI 모델과 상호작용하는 새로운 방식으로, 이전 API보다 더 간단하고 유연하게 설계되었습니다. 이 API는 여러 도구를 사용하고, 다중 턴 대화를 처리하며, 텍스트뿐만 아니라 다양한 유형의 데이터를 다룰 수 있는 고급 AI 애플리케이션을 쉽게 구축할 수 있도록 합니다.

Chat Completions와 같이 주로 텍스트를 위해 설계된 이전 API나 설정이 많이 필요한 Assistants API와 달리, Responses API는 다음을 위해 처음부터 설계되었습니다:

- 원활한 다중 턴 상호작용 (단일 API 호출에서 여러 단계의 대화를 이어갈 수 있음)
- 강력한 호스팅 도구에 대한 쉬운 접근 (파일 검색, 웹 검색, 코드 해석기 등)
- 모델에 보낼 컨텍스트에 대한 세밀한 제어

AI 모델이 점점 더 복잡하고 장기적인 추론을 수행할 수 있게 되면서, 비동기적이고 상태를 유지할 수 있는 API가 필요합니다. Responses API는 이러한 요구를 충족하도록 설계되었습니다.

이 가이드에서는 Responses API가 제공하는 새로운 기능과 함께 시작하는 데 도움이 되는 실용적인 예제를 살펴봅니다.

## 기본 사항

Responses API는 표면적으로 Completions API와 매우 유사하게 설계되었습니다.

In [ ]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2025-03-01-preview"
)

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="농담 하나 해줘",
)

In [ ]:
print(response.output[0].content[0].text)

Responses API의 주요 기능 중 하나는 상태를 유지한다는 점입니다. 즉, 대화의 상태를 직접 관리할 필요가 없으며, API가 이를 대신 처리합니다. 예를 들어, 언제든지 응답을 가져오면 전체 대화 기록이 포함됩니다.

In [ ]:
fetched_response = client.responses.retrieve(
    response_id=response.id
)

print(fetched_response.output[0].content[0].text)

이전 응답을 참조하여 대화를 계속할 수 있습니다.

In [ ]:
response_two = client.responses.create(
    model="gpt-4o-mini",
    input="다른 농담 하나 더 해줘",
    previous_response_id=response.id
)

In [ ]:
print(response_two.output[0].content[0].text)

물론 컨텍스트를 직접 관리할 수도 있습니다. 하지만 OpenAI가 컨텍스트를 유지해 주는 이점 중 하나는 응답을 원하는 시점에서 분기(fork)하여 해당 지점에서 대화를 계속할 수 있다는 점입니다.

In [ ]:
response_two_forked = client.responses.create(
    model="gpt-4o-mini",
    input="그 농담은 별로였어. 다른 농담 하나 더 해주고 두 농담의 차이점도 알려줘",
    previous_response_id=response.id  # 첫 번째 응답에서 분기하여 계속 진행
)

output_text = response_two_forked.output[0].content[0].text
print(output_text)

## 호스팅 도구

Responses API의 또 다른 이점은 `file_search` 및 `web_search`와 같은 호스팅 도구를 지원한다는 점입니다. 도구를 수동으로 호출하는 대신, 도구를 전달하면 API가 어떤 도구를 사용할지 결정하고 이를 사용합니다.

다음은 `web_search` 도구를 사용하여 웹 검색 결과를 응답에 통합하는 예제입니다.

In [ ]:
client = AzureOpenAI(
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2025-03-01-preview"
)

response = client.responses.create(
    model="gpt-4o",  # 또는 다른 지원 모델
    input="AI에 대한 최신 뉴스는 무엇인가요?",
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [ ]:
import json
print(json.dumps(response.output, default=lambda o: o.__dict__, indent=2))

## 멀티모달, 도구 확장 대화

Responses API는 텍스트, 이미지, 오디오 모달리티를 기본적으로 지원합니다.  
모든 것을 결합하여 Responses API를 통해 단일 API 호출로 완전한 멀티모달, 도구 확장 상호작용을 구축할 수 있습니다.

In [ ]:
import base64

from IPython.display import Image, display

# 제공된 URL에서 이미지를 표시
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"
display(Image(url=url, width=400))

response_multimodal = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": 
                 "이미지와 관련된 키워드를 생성하고, 검색 도구를 사용하여 키워드와 관련된 뉴스를 웹에서 검색하세요. "
                 "결과를 요약하고 출처를 인용하세요."},
                {"type": "input_image", "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"}
            ]
        }
    ],
    tools=[
        {"type": "web_search"}
    ]
)

In [ ]:
import json
print(json.dumps(response_multimodal.__dict__, default=lambda o: o.__dict__, indent=4))

위 예제에서는 `web_search` 도구를 사용하여 단일 API 호출로 이미지와 관련된 뉴스를 검색할 수 있었습니다. 이는 Chat Completions API를 사용할 경우 여러 번의 왕복 호출이 필요했던 작업을 단순화합니다.

Responses API를 사용하면  
🔥 단일 API 호출로 다음 작업을 처리할 수 있습니다:

✅ 멀티모달 입력을 사용하여 주어진 이미지를 분석합니다.

✅ `web_search` 호스팅 도구를 통해 웹 검색을 수행합니다.

✅ 결과를 요약합니다.

반면, Chat Completions API를 사용하면 여러 단계가 필요하며, 각 단계는 API로의 왕복 호출을 요구합니다:

1️⃣ 이미지를 업로드하고 분석 결과를 얻음 → 1회 요청

2️⃣ 정보를 추출하고 외부 웹 검색 호출 → 수동 단계 + 도구 실행

3️⃣ 도구 결과를 요약을 위해 다시 제출 → 추가 요청

다음 다이어그램에서 시각적으로 비교한 내용을 확인하세요!

![Responses vs Completions](../../images/comparisons.png)

Responses API를 사용해 보고 코드 단순화와 복잡한 멀티모달, 도구 확장 상호작용을 구축하는 데 얼마나 유용한지 확인해 보세요!